# QM-only Calculation 

In this example, we will use QMzyme to create a truncated active site composed of residues within 4 Angstroms of the bound ligand. All C-alpha atoms are set to be frozen during geometry optimization.

Classes used in this example:

- [GenerateModel](https://qmzyme.readthedocs.io/en/latest/API/QMzyme.GenerateModel.html)

- [QM_Method](https://qmzyme.readthedocs.io/en/latest/API/QMzyme.CalculateModel.html#qm-treatment)

- [CA_terminal TruncationScheme](https://qmzyme.readthedocs.io/en/latest/API/QMzyme.TruncationSchemes.html#QMzyme.TruncationSchemes.CA_terminal)

- [DistanceCutoff SelectionScheme](https://qmzyme.readthedocs.io/en/latest/API/QMzyme.SelectionSchemes.html#QMzyme.SelectionSchemes.DistanceCutoff)


In [2]:
import QMzyme 
from QMzyme.data import PDB

#### Initialize Model and Define Regions

In [3]:
model = QMzyme.GenerateModel(PDB)

# add unknown residue charge
QMzyme.data.residue_charges.update({'EQU': -1})

# set catalytic center
model.set_catalytic_center(selection='resid 263')

# import selection scheme 'DistanceCutoff' and use it to create 5 Angstrom 
# region around catalytic center.
from QMzyme.SelectionSchemes import DistanceCutoff
model.set_region(selection=DistanceCutoff, cutoff=5)
print(model.regions)


Charge information not present. QMzyme will try to guess region charges based on residue names consistent with AMBER naming conventions (i.e., aspartate: ASP --> Charge: -1, aspartic acid: ASH --> Charge: 0.). See QMzyme.data.residue_charges for the full set.

	Nonconventional Residues Found
	------------------------------
	EQU --> Charge: UNK, defaulting to 0

You can update charge information for nonconventional residues by running 
	>>>QMzyme.data.residue_charges.update({'3LETTER_RESNAME':INTEGER_CHARGE}). 
Note your changes will not be stored after you exit your session. It is recommended to only alter the residue_charges dictionary. If you alter the protein_residues dictionary instead that could cause unintended bugs in other modules (TruncationSchemes).

[<QMzymeRegion catalytic_center contains 37 atom(s) and 1 residue(s)>, <QMzymeRegion cutoff_5 contains 427 atom(s) and 33 residue(s)>]


#### Designate Coordinate Constraints

In [4]:
c_alpha_atoms = model.cutoff_5.get_atoms(attribute='name', value='CA')
model.cutoff_5.set_fixed_atoms(atoms=c_alpha_atoms)

print("Fixed atoms: ",model.cutoff_5.get_atoms(attribute='is_fixed', value=True))

Fixed atoms:  [<QMzymeAtom 235: CA of resname TYR, resid 16>, <QMzymeAtom 256: CA of resname ILE, resid 17>, <QMzymeAtom 309: CA of resname VAL, resid 20>, <QMzymeAtom 595: CA of resname ASP, resid 40>, <QMzymeAtom 615: CA of resname PRO, resid 41>, <QMzymeAtom 851: CA of resname TYR, resid 57>, <QMzymeAtom 896: CA of resname GLN, resid 59>, <QMzymeAtom 913: CA of resname GLY, resid 60>, <QMzymeAtom 920: CA of resname LEU, resid 61>, <QMzymeAtom 982: CA of resname VAL, resid 66>, <QMzymeAtom 1022: CA of resname ALA, resid 68>, <QMzymeAtom 1225: CA of resname MET, resid 84>, <QMzymeAtom 1256: CA of resname PHE, resid 86>, <QMzymeAtom 1300: CA of resname VAL, resid 88>, <QMzymeAtom 1331: CA of resname MET, resid 90>, <QMzymeAtom 1461: CA of resname LEU, resid 99>, <QMzymeAtom 1492: CA of resname VAL, resid 101>, <QMzymeAtom 1527: CA of resname ASH, resid 103>, <QMzymeAtom 1743: CA of resname MET, resid 116>, <QMzymeAtom 1777: CA of resname ALA, resid 118>, <QMzymeAtom 1808: CA of resname

#### Build the QM Method and Assign to Region

In [5]:
qm_method = QMzyme.QM_Method(
    basis_set='6-31G*', 
    functional='wB97X-D3', 
    qm_input='OPT FREQ', 
    program='orca'
)

# since we are not specifying the charge in this method below, the method
# will estimate the charge based on residue naming conventions
qm_method.assign_to_region(region=model.cutoff_5)

# you would alternatively uncomment and run:
#qm_method.assign_to_region(region=model.cutoff_5, charge=-2, mult=1)


Estimating total charge for QMzymeRegion cutoff_5 based on residue naming conventions in QMzyme.data.residue_charges...
	<QMzymeResidue resname: TYR, resid: 16, chain: A> --> Charge: 0
	<QMzymeResidue resname: ILE, resid: 17, chain: A> --> Charge: 0
	<QMzymeResidue resname: VAL, resid: 20, chain: A> --> Charge: 0
	<QMzymeResidue resname: ASP, resid: 40, chain: A> --> Charge: -1
	<QMzymeResidue resname: PRO, resid: 41, chain: A> --> Charge: 0
	<QMzymeResidue resname: TYR, resid: 57, chain: A> --> Charge: 0
	<QMzymeResidue resname: GLN, resid: 59, chain: A> --> Charge: 0
	<QMzymeResidue resname: GLY, resid: 60, chain: A> --> Charge: 0
	<QMzymeResidue resname: LEU, resid: 61, chain: A> --> Charge: 0
	<QMzymeResidue resname: VAL, resid: 66, chain: A> --> Charge: 0
	<QMzymeResidue resname: ALA, resid: 68, chain: A> --> Charge: 0
	<QMzymeResidue resname: MET, resid: 84, chain: A> --> Charge: 0
	<QMzymeResidue resname: PHE, resid: 86, chain: A> --> Charge: 0
	<QMzymeResidue resname: VAL, res

#### Truncate Model

In [6]:
model.truncate()
print("Truncated model: ", QMzyme.CalculateModel.calculation['QM'])
print("Method details: ", QMzyme.CalculateModel.calculation['QM'].method)


Truncated model has been created and saved to attribute 'truncated' and stored in QMzyme.CalculateModel.calculation under key QM. This model will be used to write the calculation input.
Truncated model:  <QMzymeRegion cutoff_5_truncated contains 391 atom(s) and 33 residue(s)>
Method details:  {'type': 'QM', 'qm_input': '6-31G* wB97X-D3 OPT FREQ', 'basis_set': '6-31G*', 'functional': 'wB97X-D3', 'qm_end': '', 'program': 'orca', 'freeze_atoms': [2, 23, 42, 58, 78, 84, 105, 122, 129, 148, 164, 174, 191, 211, 227, 244, 263, 279, 292, 309, 319, 343], 'mult': 1, 'charge': -2}


#### Write QM Input File

In [7]:
model.write_input()

File /Users/hrk/git/QMzyme/docs/Examples/QCALC/cutoff_5_truncated.inp created.
